In [ ]:
import json
import numpy as np
import glob
import PIL.Image
import os
from PIL import ImageDraw
from shapely.geometry import Polygon

class labelme2coco(object):
    def __init__(self, labelme_json=[], save_json_path='./new.json',categories_path='./categories.json'):
        '''
        :param labelme_json: 所有labelme的json文件路径组成的列表
        :param save_json_path: json保存位置
        '''
        self.labelme_json = labelme_json
        self.save_json_path = save_json_path
        self.annotations = []
        self.images = []
        self.categories = self.load_categories(categories_path) # 指定标注的类别
        with open(".\custom_coco\categories.json", 'w') as f:
            json.dump(self.categories, f)
        self.label = []
        self.annID = 1
        self.height = 0
        self.width = 0
        self.save_json()
    
    def load_categories(self, categories_path):
        categories = []
        if categories_path and os.path.exists(categories_path):
            with open(categories_path, 'r') as f:
                categories = json.load(f)
            print(f"Loaded {len(categories)} categories from {categories_path}.")
        id=len(categories)
        new_categories = [
            {'supercategory': 'none', 'color':[220,20,90],'isthing':1,'id': 201, 'name': 'digger'},
            {'supercategory': 'none', 'color':[240,30,40],'isthing':1,'id': 202, 'name': 'lorry'},
            {'supercategory': 'none', 'color':[240,0,70],'isthing':1,'id': 203, 'name': 'bulldozer'},
            {'supercategory': 'none', 'color':[180,75,25],'isthing':1,'id': 204, 'name': 'netbu'},
            {'supercategory': 'none', 'color':[200,90,150],'isthing':1,'id': 205, 'name': 'trunk'},
            {'supercategory': 'none', 'color':[75,150,60],'isthing':1,'id': 206, 'name': 'greenhouse'},
            {'supercategory': 'none', 'color':[90,120,120],'isthing':1,'id': 207, 'name': 'smoggy'},
            {'supercategory': 'none', 'color':[160,200,100],'isthing':1,'id': 208, 'name': 'toweldiao'},
            {'supercategory': 'none', 'color':[10,200,20],'isthing':1, 'id': 209, 'name': 'fish'},
        ]
        categories.extend(new_categories)
        return categories
    # 定义读取图像标注信息的方法
    def image(self, data, num):
        image = {}
        height = data['imageHeight']
        width = data['imageWidth']
        image['height'] = height
        image['width'] = width
        image['id'] = num + 1
        image['file_name'] = data['imagePath'].split('/')[-1]
        self.height = height
        self.width = width
        return image

    # 定义数据转换方法
    def data_transfer(self):
        for num, json_file in enumerate(self.labelme_json):
            with open(json_file, 'r') as fp:
                data = json.load(fp)  # 加载json文件
                self.images.append(self.image(data, num)) # 读取所有图像标注信息并加入images数组
                for shapes in data['shapes']:
                    label = shapes['label']
                    points = shapes['points']
                    shape_type = shapes['shape_type']
                    if shape_type == 'rectangle':
                        points = [points[0],[points[0][0],points[1][1]],points[1],[points[1][0],points[0][1]]]     
                    self.annotations.append(self.annotation(points, label, num)) # 读取所有检测框标注信息并加入annotations数组
                    self.annID += 1
        print(self.annotations)

    # 定义读取检测框标注信息的方法
    def annotation(self, points, label, num):
        annotation = {}
        annotation['segmentation'] = [list(np.asarray(points).flatten())]
        poly = Polygon(points)
        area_ = round(poly.area, 6)
        annotation['area'] = area_
        annotation['iscrowd'] = 0
        annotation['image_id'] = num + 1
        annotation['bbox'] = list(map(float, self.getbbox(points)))
        annotation['category_id'] = self.getcatid(label)
        annotation['id'] = self.annID
        return annotation

    # 定义读取检测框的类别信息的方法
    def getcatid(self, label):
        for categorie in self.categories:
            if label == categorie['name']:
                return categorie['id']
        return -1

    def getbbox(self, points):
        polygons = points
        mask = self.polygons_to_mask([self.height, self.width], polygons)
        return self.mask2box(mask)

    def mask2box(self, mask):
        '''从mask反算出其边框
        mask：[h,w]  0、1组成的图片
        1对应对象，只需计算1对应的行列号（左上角行列号，右下角行列号，就可以算出其边框）
        '''
        # np.where(mask==1)
        index = np.argwhere(mask == 1)
        rows = index[:, 0]
        clos = index[:, 1]
        # 解析左上角行列号
        left_top_r = np.min(rows)  # y
        left_top_c = np.min(clos)  # x

        # 解析右下角行列号
        right_bottom_r = np.max(rows)
        right_bottom_c = np.max(clos)

        return [left_top_c, left_top_r, right_bottom_c - left_top_c,
                right_bottom_r - left_top_r]  # [x1,y1,w,h] 对应COCO的bbox格式

    def polygons_to_mask(self, img_shape, polygons):
        mask = np.zeros(img_shape, dtype=np.uint8)
        mask = PIL.Image.fromarray(mask)
        xy = list(map(tuple, polygons))
        PIL.ImageDraw.Draw(mask).polygon(xy=xy, outline=1, fill=1)
        mask = np.array(mask, dtype=bool)
        return mask

    def data2coco(self):
        data_coco = {}
        data_coco['images'] = self.images
        data_coco['categories'] = self.categories
        data_coco['annotations'] = self.annotations
        return data_coco

    def save_json(self):
        self.data_transfer()
        self.data_coco = self.data2coco()
        # 保存json文件
        json.dump(self.data_coco, open(self.save_json_path, 'w'), indent=4)  # 写入指定路径的json文件，indent=4 更加美观显示

labelme_json = glob.glob('./your_file_path/*.json')  # 获取指定目录下的json格式的文件
output_dir = './your_output_dir'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
labelme2coco(labelme_json, './annotations/instance_train.json','./panoptic_coco_categories.json') # 指定生成文件路径

Loaded 133 categories from ./panopticapi/panoptic_coco_categories.json.
[{'segmentation': [[361.9047619047619, 1465.4523809523812, 394.04761904761904, 1503.5476190476193, 370.23809523809524, 1534.5000000000002, 282.14285714285717, 1523.7857142857144, 278.57142857142856, 1458.309523809524, 291.6666666666667, 1445.2142857142858, 375.0, 1244.0238095238096, 515.4761904761905, 1179.7380952380954, 580.952380952381, 1229.7380952380954, 609.5238095238095, 1210.6904761904764, 642.8571428571429, 1205.9285714285716, 717.8571428571429, 1227.3571428571431, 722.6190476190476, 1277.3571428571431, 695.2380952380953, 1297.5952380952383, 698.8095238095239, 1319.0238095238096, 617.8571428571429, 1407.1190476190477, 592.8571428571429, 1409.5000000000002, 589.2857142857143, 1373.7857142857144, 542.8571428571429, 1363.0714285714287, 494.0476190476191, 1407.1190476190477, 464.2857142857143, 1407.1190476190477, 458.33333333333337, 1377.3571428571431, 511.9047619047619, 1321.4047619047622, 498.80952380952385, 